In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2024-03-18 23:09:07.708461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 23:09:07.708536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 23:09:07.711280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 23:09:07.729732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 23:09:10.106105: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels

# Get list of image paths
image_paths = list(paths.list_images("../BrainTumor/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/2870
[INFO]: Processed 2000/2870
(2870, 224, 224, 3)
(2870,)


In [8]:
num_classes = len(np.unique(labels))

In [9]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        data = data.reshape(data.shape[0],data.shape[1] * data.shape[2] * data.shape[3])
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [10]:
from sklearn.linear_model import LogisticRegression
# LR
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)


[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.759581881533101
Sensitivity: 0.7503984041230588
Specificity: 0.918024154980639
AUC-ROC: 0.9144690522055262
MCC: 0.6718198287020263
Precision: 0.7554306423786157
F1 Score: 0.752680715157263

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7804878048780488
Sensitivity: 0.7615273476645795
Specificity: 0.9259516591845236
AUC-ROC: 0.8806727626760051
MCC: 0.70162215751507
Precision: 0.7588448907414425
F1 Score: 0.7585807581443857

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.7491289198606271
Sensitivity: 0.74946738172201
Specificity: 0.9141312923905613
AUC-ROC: 0.9085211534130704
MCC: 0.6583133588437874
Precision: 0.7464733848500035
F1 Score: 0.7476593595144013

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.794425087108014
Sensitivity: 0.7888811342932707
Specificity: 0.9294046975954399
AUC-ROC: 0.9183370963579172
MCC: 0.7196826000866425
Precision: 0.7977951102951103
F1 Score: 0.7925962860976573

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.759581881533101
Sensitivity: 0.7327101087275933
Specificity: 0.9175705065278802
AUC-ROC: 0.9122886934503993
MCC: 0.6709814390469694
Precision: 0.7506054884507642
F1 Score: 0.7387631180146941


In [11]:
# SVM
#svm_model = SVC(probability=True)
#evaluate_model(svm_model, 'SVM', data, labels)


#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.578397212543554
Sensitivity: 0.5478320411100311
Specificity: 0.8540035250921768
AUC-ROC: 0.700917783101104
MCC: 0.44868139198977697
Precision: 0.5380835543766579
F1 Score: 0.5095498237444716

[INFO] Fold 2 / 10 for NB
Accuracy: 0.5679442508710801
Sensitivity: 0.5591116812465622
Specificity: 0.8529361626633345
AUC-ROC: 0.7048787702595288
MCC: 0.43814451701795426
Precision: 0.5061128035606257
F1 Score: 0.5086769159096203

[INFO] Fold 3 / 10 for NB
Accuracy: 0.6167247386759582
Sensitivity: 0.6150935825855015
Specificity: 0.8674360932413878
AUC-ROC: 0.7443407270780434
MCC: 0.5025102764229787
Precision: 0.6186002178649238
F1 Score: 0.5789359541798015

[INFO] Fold 4 / 10 for NB
Accuracy: 0.6027874564459931
Sensitivity: 0.5918034238266386
Specificity: 0.862669408833557
AUC-ROC: 0.7261142807288694
MCC: 0.4870609714915547
Precision: 0.5750333166749958
F1 Score: 0.550478241478131

[INFO] Fold 5 / 10 for NB
Accuracy: 0.6306620209059234
Sensitivity: 0.6228431

In [12]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7979094076655052
Sensitivity: 0.7900702246132748
Specificity: 0.9302859027167123
AUC-ROC: 0.9523923085754715
MCC: 0.7293654897870671
Precision: 0.8068053663138036
F1 Score: 0.7900845294148989

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.8153310104529616
Sensitivity: 0.7983707814371934
Specificity: 0.9381288471328737
AUC-ROC: 0.9423030602090307
MCC: 0.7551295985035361
Precision: 0.8026529482216515
F1 Score: 0.7907525113598216

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.8083623693379791
Sensitivity: 0.8025413097041072
Specificity: 0.9337449476249982
AUC-ROC: 0.9390395103361894
MCC: 0.7434781823428044
Precision: 0.8197550628049435
F1 Score: 0.8032628101869546

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.794425087108014
Sensitivity: 0.7812902642464794
Specificity: 0.9292098625445457
AUC-ROC: 0.9404079679483864
MCC: 0.727866717970588
Precision: 0.8086011626403303
F1 Score: 0.7830175653362181

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7560975609756098
Sensitivity: 0.73

In [13]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.8989547038327527
Sensitivity: 0.9021621193969122
Specificity: 0.9662958377686244
AUC-ROC: 0.9866227540070278
MCC: 0.8646398650901709
Precision: 0.8945730597904511
F1 Score: 0.8951889175594337

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.89198606271777
Sensitivity: 0.8991321759834837
Specificity: 0.9653309691303744
AUC-ROC: 0.9849897460250068
MCC: 0.855889683620612
Precision: 0.8750940839981937
F1 Score: 0.8819567344665815

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.867595818815331
Sensitivity: 0.8544440425868578
Specificity: 0.9559246810447232
AUC-ROC: 0.9801397469882283
MCC: 0.820131397883858
Precision: 0.8530043486481376
F1 Score: 0.8528747033683363

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.8850174216027874
Sensitivity: 0.8830603841256206
Specificity: 0.9606579079310718
AUC-ROC: 0.9839696386883388
MCC: 0.8450832374237696
Precision: 0.8898637929518922
F1 Score: 0.883961618226126

[INFO] Fold 5 / 10 for Random Forest


In [14]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 29.796524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21000372
[LightGBM] [Info] Number of data points in the train set: 2583, number of used features: 150528
[LightGBM] [Info] Start training from score -1.246011
[LightGBM] [Info] Start training from score -1.250057
[LightGBM] [Info] Start training from score -1.981776
[LightGBM] [Info] Start training from score -1.244666
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.9059233449477352
Sensitivity: 0.9081494834874208
Specificity: 0.9683059887999259
AUC-ROC: 0.9889542034787013
MCC: 0.8731108181105262
Precision: 0.9029629275952806
F1 Score: 0.9038059847594953

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 29.925221 seconds.
You can set `force_col_wise=true` to remove t

In [15]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the VGG16 model pre-trained on ImageNet data
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Build a new model using VGG16 as a feature extractor
with tf.device('/GPU:0'):
    model_vgg16 = Sequential()
    model_vgg16.add(vgg16_model)
    model_vgg16.add(Flatten())
    model_vgg16.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_vgg16.add(Dropout(0.5, seed = 0))
    model_vgg16.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_vgg16, 'VGG16', data, labels)

2024-03-19 05:14:56.282140: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8447 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0



[INFO] Fold 1 / 10 for VGG16


2024-03-19 05:14:58.406950: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20


2024-03-19 05:15:03.540223: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-19 05:15:03.853458: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 05:15:07.026338: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f43eb720e80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-19 05:15:07.026399: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-03-19 05:15:07.041914: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710825307.351275  956491 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 [==============================] - 9s 54ms/step - loss: 9.1348 - accuracy: 0.6860
Epoch 2/20
41/41 [==============================] - 2s 46ms/step - loss: 0.6287 - accuracy: 0.8076
Epoch 3/20
41/41 [==============================] - 2s 46ms/step - loss: 0.4398 - accuracy: 0.8405
Epoch 4/20
41/41 [==============================] - 2s 46ms/step - loss: 0.3681 - accuracy: 0.8606
Epoch 5/20
41/41 [==============================] - 2s 46ms/step - loss: 0.3157 - accuracy: 0.8839
Epoch 6/20
41/41 [==============================] - 2s 46ms/step - loss: 0.2320 - accuracy: 0.9110
Epoch 7/20
41/41 [==============================] - 2s 46ms/step - loss: 0.2061 - accuracy: 0.9210
Epoch 8/20
41/41 [==============================] - 2s 46ms/step - loss: 0.2024 - accuracy: 0.9319
Epoch 9/20
41/41 [==============================] - 2s 44ms/step - loss: 0.2288 - accuracy: 0.9187
Epoch 10/20
41/41 [==============================] - 2s 44ms/step - loss: 0.2322 - accuracy: 0.9175
Epoch 11/20
41/41 [=

In [16]:

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the EfficientNetB7 model pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Build a new model using EfficientNetB7 as a feature extractor
with tf.device('/GPU:0'):
    model_efficientnet = Sequential()
    model_efficientnet.add(efficientnet_model)
    model_efficientnet.add(Flatten())
    model_efficientnet.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_efficientnet.add(Dropout(0.5, seed=0))
    model_efficientnet.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_efficientnet, 'EfficientNetB0', data, labels)



[INFO] Fold 1 / 10 for EfficientNetB0
Epoch 1/20
41/41 [==============================] - 19s 53ms/step - loss: 3.9943 - accuracy: 0.6411
Epoch 2/20
41/41 [==============================] - 2s 44ms/step - loss: 0.5364 - accuracy: 0.7755
Epoch 3/20
41/41 [==============================] - 2s 43ms/step - loss: 0.4177 - accuracy: 0.8250
Epoch 4/20
41/41 [==============================] - 2s 43ms/step - loss: 0.3608 - accuracy: 0.8432
Epoch 5/20
41/41 [==============================] - 2s 44ms/step - loss: 0.2974 - accuracy: 0.8908
Epoch 6/20
41/41 [==============================] - 2s 39ms/step - loss: 0.3105 - accuracy: 0.8719
Epoch 7/20
41/41 [==============================] - 2s 44ms/step - loss: 0.2593 - accuracy: 0.8862
Epoch 8/20
41/41 [==============================] - 2s 43ms/step - loss: 0.2349 - accuracy: 0.8955
Epoch 9/20
41/41 [==============================] - 2s 43ms/step - loss: 0.2274 - accuracy: 0.9032
Epoch 10/20
41/41 [==============================] - 2s 44ms/step - l

In [17]:
from tensorflow.keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the ResNet50 model pre-trained on ImageNet data
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build a new model using ResNet50 as a feature extractor
with tf.device('/GPU:0'):
    model_resnet50 = Sequential()
    model_resnet50.add(resnet50_model)
    model_resnet50.add(Flatten())
    model_resnet50.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_resnet50.add(Dropout(0.5, seed=0))
    model_resnet50.add(Dense(num_classes, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_resnet50, 'ResNet50', data, labels)



[INFO] Fold 1 / 10 for ResNet50
Epoch 1/20
41/41 [==============================] - 12s 66ms/step - loss: 12.1546 - accuracy: 0.6009
Epoch 2/20
41/41 [==============================] - 2s 52ms/step - loss: 0.7307 - accuracy: 0.6802
Epoch 3/20
41/41 [==============================] - 2s 53ms/step - loss: 0.6413 - accuracy: 0.7007
Epoch 4/20
41/41 [==============================] - 2s 53ms/step - loss: 0.5798 - accuracy: 0.7499
Epoch 5/20
41/41 [==============================] - 2s 52ms/step - loss: 0.5177 - accuracy: 0.7704
Epoch 6/20
41/41 [==============================] - 2s 53ms/step - loss: 0.4910 - accuracy: 0.7755
Epoch 7/20
41/41 [==============================] - 2s 52ms/step - loss: 0.4411 - accuracy: 0.8099
Epoch 8/20
41/41 [==============================] - 2s 52ms/step - loss: 0.4247 - accuracy: 0.8161
Epoch 9/20
41/41 [==============================] - 2s 52ms/step - loss: 0.3882 - accuracy: 0.8308
Epoch 10/20
41/41 [==============================] - 2s 52ms/step - loss: 

In [18]:
# Save results to a CSV file
results_df.to_csv('DS5_baseline.csv', index=False)

In [19]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.766551,0.756230,0.920093,0.907396,0.681493,0.762434,0.758225,416.191406,2894.500581
1,NB,0.602091,0.592826,0.863132,0.728495,0.486804,0.579902,0.551047,371.125000,173.700305
2,KNN,0.802787,0.787635,0.932723,0.944165,0.738224,0.803705,0.784896,376.613281,119.695549
3,Random Forest,0.894077,0.895515,0.964637,0.983295,0.857312,0.888012,0.889447,413.484375,122.244015
4,LGBM,0.907666,0.909982,0.969286,0.986747,0.875420,0.900003,0.903058,8644.351562,18597.837821
5,VGG16,0.990244,0.990854,0.996623,0.999536,0.986895,0.991405,0.990896,934.636719,174.985255
6,EfficientNetB0,0.985714,0.986895,0.995117,0.999463,0.980618,0.985942,0.986260,292.871094,166.702671
7,ResNet50,0.989199,0.989928,0.996299,0.999291,0.985374,0.989556,0.989615,161.539062,230.242539
